In [1]:
#dumb installs
%load_ext autoreload
%autoreload 2

import os
import astrophot as ap
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
from random import randint
from pandas import read_csv
from PIL import Image
%matplotlib inline

In [ ]:
img_name = os.listdir("galaxyZoo/images/")
for img in img_name:
    image = Image.open(f"galaxyZoo/images/{img}")
    image = image.convert('L')
    image = np.array(image)
    hdu = fits.PrimaryHDU(image)
    hdu.writeto(f"galaxyZoo/fits_images/f_{img.removesuffix('.jpg')}.fits", overwrite=True)

In [ ]:
final_df = read_csv(os.path.join("galaxyZoo/please.csv")) #csv file containing the classification associated with each SDSS object
spec_list = []
class_df = final_df["gz2_class"]
for n in class_df:
    spec_list.append(n)
spec_list = [s[:2] for s in spec_list] #cuts all the weird stuff from the classification
#batch because laptop doesn't have enough memory
spec_list = spec_list[:4855]

In [ ]:
img_name = os.listdir("galaxyZoo/fits_images/")
count = 0
params_all = []
spiral = []
elliptical = []

for img in img_name:
    #making a target (basically data for the model to fit)
    file = fits.open(f"galaxyZoo/fits_images/{img}")
    target_data = np.array(file[0].data, dtype = np.float64)

    # target_data = target_data.T

    #target object
    target = ap.image.Target_Image(
        data = target_data,
        pixelscale = 0.396, #SDSS pixel scale
        zeropoint = 22.3, #r-band in SDSS
        variance = np.ones(target_data.shape)/1e3,
    )
    model1 = ap.models.AstroPhot_Model(
    name = f"model{count}", # every model must have a unique name
    model_type = "sersic galaxy model", # this specifies the kind of model
    #we don't need to list params because we have a target !
    target = target,
    )
    model1.initialize()
    result = ap.fit.lm.LM(model1, verbose = 0).fit() #verbose is 0 by default, the higher, the more detailed output
    dict = model1.get_state()
    n = dict["parameters"]["nodes"][3]["value"]
    
    classification = spec_list[count]
    if classification[0] == "S":
        spiral.append(n)
    elif classification[0] == "E":
        elliptical.append(n)
    params_all.append(n)

    print(count)
    count += 1

In [7]:
print(f"The average sersic index of spiral galaxies is {sum(spiral) / len(spiral)}")
print(f"The average sersic index of elliptical galaxies is {sum(elliptical) / len(elliptical)}")
print(f"The average sersic index is {sum(params_all) / len(params_all)}")

The average sersic index of spiral galaxies is 1.5358194919149737
The average sersic index of elliptical galaxies is 1.5810735722722504
The average sersic index is 1.555014241248085


In [9]:
print(f"The range of spiral was {max(spiral) - min(spiral)}")
print(f"min: {max(spiral)}, max: {min(spiral)}")

print(f"The range of elliptical was {max(elliptical) - min(elliptical)}")
print(f"min: {max(elliptical)}, max: {min(elliptical)}")

print(f"The range of all was {max(params_all) - min(params_all)}")
print(f"min: {max(params_all)}, max: {min(params_all)}")

The range of spiral was 7.639999996958935
min: 7.999999997152622, max: 0.36000000019368594
The range of elliptical was 7.6399933244599385
min: 7.999993324643386, max: 0.36000000018344724
The range of all was 7.639999996969174
min: 7.999999997152622, max: 0.36000000018344724
